In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 

In [2]:
import earth_model

## Tabulate the model

And write in Mineos format

In [3]:
prem = earth_model.Prem()
# Get a table of PREM values every 10 km going inwards
# and dealing with the discontiuities 
table = prem.tabulate_model_outwards(25)
# Print the firs few depths
# note the discontiuities
print(np.record.pprint(table[0:7]))
print()
print(np.record.pprint(table[-8:-1]))
print(table.shape)

  depth: [6371. 6346. 6321. 6296. 6271. 6246. 6221.]
 radius: [  0.  25.  50.  75. 100. 125. 150.]
density: [13088.5        13088.36391078 13087.95564313 13087.27519704
 13086.32257251 13085.09776955 13083.60078815]
     vp: [11.2622     11.26210201 11.26180803 11.26131806 11.26063211 11.25975018
 11.25867225]
     vs: [3.6678     3.66773152 3.66752607 3.66718366 3.66670428 3.66608793
 3.66533462]
 qkappa: [1327.7 1327.7 1327.7 1327.7 1327.7 1327.7 1327.7]
 qshear: [84.6 84.6 84.6 84.6 84.6 84.6 84.6]

  depth: [95.  80.  55.  30.  24.4 24.4 15. ]
 radius: [6276.  6291.  6316.  6341.  6346.6 6346.6 6356. ]
density: [3373.07540418 3374.70560352 3377.42260242 3380.13960132 3380.74820907
 2900.         2900.        ]
     vp: [8.06697625 8.07624842 8.09170204 8.10715566 8.11061727 6.8
 6.8       ]
     vs: [4.46498674 4.47051515 4.47972916 4.48894318 4.49100712 3.9
 3.9       ]
 qkappa: [57823. 57823. 57823. 57823. 57823. 57823. 57823.]
 qshear: [ 80. 600. 600. 600. 600. 600. 600.]
(268,)

In [4]:
model_name = 'model'
mineos_model_filename = model_name + '.txt'
f = open(mineos_model_filename, 'w')
f.write("Model_name_goes_here\n")
f.write("   0    1.00000  1\n") # isotropic tabulated format 1s ref
n = table.shape[0]
nic = np.argwhere(table.vs==0)[0][0] # This is the last inner core index. 
noc = np.argwhere(table.vs==0)[-1][0] + 1 # This is the last outer core index (assuming no ocean).
f.write("{} {} {}\n".format(n, nic, noc))
for r, rho, vp, vs, qk, qm in zip(table.radius, table.density, table.vp, table.vs, 
                          table.qkappa, table.qshear):
    f.write("{:#8.0f} {:#9.2f} {:#9.2f} {:#9.2f} {:#9.1f} {:#9.1f} {:#9.2f} {:#9.2f} {:#9.5f}\n".format(r*1000.0, rho, 
                                                               vp*1000.0, vs*1000.0,
                                                               qk, qm, vp*1000.0, vs*1000.0, 1.0))

f.close()

In [5]:
!head "$mineos_model_filename"

Model_name_goes_here
   0    1.00000  1
268 50 142
      0.  13088.50  11262.20   3667.80    1327.7      84.6  11262.20   3667.80   1.00000
  25000.  13088.36  11262.10   3667.73    1327.7      84.6  11262.10   3667.73   1.00000
  50000.  13087.96  11261.81   3667.53    1327.7      84.6  11261.81   3667.53   1.00000
  75000.  13087.28  11261.32   3667.18    1327.7      84.6  11261.32   3667.18   1.00000
 100000.  13086.32  11260.63   3666.70    1327.7      84.6  11260.63   3666.70   1.00000
 125000.  13085.10  11259.75   3666.09    1327.7      84.6  11259.75   3666.09   1.00000
 150000.  13083.60  11258.67   3665.33    1327.7      84.6  11258.67   3665.33   1.00000


In [6]:
!cat mineos_param.txt

model.txt
prem_noocean_S
eprem_noocean_S
1e-10 10
3
1 250 0.0 166.0 0 0


In [7]:
!../cig/bin/minos_bran < mineos_param.txt

 input model file:
 model.txt
 output file:
 prem_noocean_S
 eigenfunction file (output):
 eprem_noocean_S
 enter eps and wgrav
   1.0000000000000000E-010   10.000000000000000     
 enter jcom (1=rad;2=tor;3=sph;4=ictor)
           3
 enter lmin,lmax,wmin,wmax,nmin,nmax
           1         250   0.0000000000000000        166.00000000000000                0           0


In [8]:
!cat prem_noocean_S

 Model_name_goes_here                                                             ref per =   1.0 secs


  level    radius        rho         vpv         vph         vsv         vsh         eta         qmu         qkap

     1         0.0    13088.50    11262.20    11262.20     3667.80     3667.80     1.00000       84.60     1327.70
     2     25000.0    13088.36    11262.10    11262.10     3667.73     3667.73     1.00000       84.60     1327.70
     3     50000.0    13087.96    11261.81    11261.81     3667.53     3667.53     1.00000       84.60     1327.70
     4     75000.0    13087.28    11261.32    11261.32     3667.18     3667.18     1.00000       84.60     1327.70
     5    100000.0    13086.32    11260.63    11260.63     3666.70     3666.70     1.00000       84.60     1327.70
     6    125000.0    13085.10    11259.75    11259.75     3666.09     3666.09     1.00000       84.60     1327.70
     7    150000.0    13083.60    11258.67    11258.67     3665.33     3665.33     1.00000

In [9]:
# Note that 0S2 is 3233.115 secs in Table V